# Solving a complex task with a multi-agent hierarchy

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [1]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 14.2 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [11]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [12]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [13]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [14]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(provider="serpapi"), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [15]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: Millennium Mills, Royal Victoria Dock, East London. The Batman location: Lt Gordon and
The Batman interrogate Penguin after the ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [Category:Film Locations - Batman Wiki](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki | Fandom

A · Arctic World · Arkham Asylum (Nolanverse) · Arkham Asylum (Schumacherverse) · Arkham State Hospital ...

5. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

6. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

7. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

8. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOopjfkJvYecQNFzxnPCBdnhxclZxuGeEmXi7ncu
u9CofWVDnJYeF)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

9. [The Filming Locations of The Batman : 
r/Tartaria](https://www.reddit.com/r/Tartaria/comments/tsd5zf/the_filming_locations_of_the_batman/)
Source: Reddit · r/Tartaria

The Filming Locations of The Batman · r/Tartaria - Royal Liver Building, Liverpool. Royal Liver Building, Liverpool
· r/Tartaria - Royal Liver ...

Out: None

[Step 1: Duration 6.88 seconds| Input tokens: 2,235 | Output tokens: 149]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extract supercar factory locations and their coordinates                                                       
  supercar_locations = [                                                                                           
      ("Ferruccio Lamborghini Museum, Funo di Argelato, Bologna, Italy", (44.6333, 11.1833)),                      
      ("Lamborghini Museum 'MUDETEC', Sant'Agata Bolognese, Italy", (44.5403, 11.3467)),                           
      ("Lamborghini Factory, Sant'Agata Bolognese, Italy", (44.5403, 11.3467)),                                    
      ("Ferrari Gallery, Maranello, Italy", (44.5646, 10.9941)),                                                   
      ("Pagani Automobili, San Cesario sul Panaro, Italy", (44.6500, 10.9667)),                                    
      ("SSC North America, Richland, Washington, USA", (46.1636, -119.2750))                                       
  ]                                                                                                                
                                                                                                                   
  # Calculate travel time for each supercar factory                                                                
  supercar_travel_times = []                                                                                       
  for location, coords in supercar_locations:                                                                      
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      supercar_travel_times.append((location, travel_time))                                                        
                                                                                                                   
  # Create DataFrame for supercar factories                                                                        
  supercar_df = pd.DataFrame(supercar_travel_times, columns=['Location', 'Travel Time (hours)'])                   
                                                                                                                   
  # Print Supercar DataFrame                                                                                       
  print(supercar_df)                                                                                               
                                                                                                                   
  # Combine Batman and Supercar DataFrames                                                                         
  combined_df = pd.concat([batman_df, supercar_df], ignore_index=True)                                             
                                                                                                                   
  # Print Combined DataFrame                                                                                       
  print(combined_df)                                                                                               
                                                                                                                   
  # Return the final answer                                                                                        
  final_answer(combined_df)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                            Location  Travel Time (hours)
0  Ferruccio Lamborghini Museum, Funo di Argelato...                10.75
1  Lamborghini Museum 'MUDETEC', Sant'Agata Bolog...                10.77
2   Lamborghini Factory, Sant'Agata Bolognese, Italy                10.77
3                  Ferrari Gallery, Maranello, Italy                10.73
4   Pagani Automobili, San Cesario sul Panaro, Italy                10.72
5       SSC North America, Richland, Washington, USA                 6.36
                                             Location  Travel Time (hours)
0          Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
1           St. George's Hall, Liverpool, England, UK                 8.81
2   Millennium Mills, Royal Victoria Dock, East Lo...                 9.18
3   Knebworth House, Knebworth, Hertfordshire, Eng...                 9.14
4   Acton Lane Power Station, Acton Lane, Acton, L...                 9.16
5                                        Arctic World                 9.83
6                          Arkham Asylum (Nolanverse)                 1.45
7                               Arkham State Hospital                 1.45
8                       Warner Bros studios, New York                 1.01
9                     Warner Bros studios, Pittsburgh                 1.74
10                                            Chicago                 2.68
11                                   Bedfordshire, UK                 9.10
12                                          Hong Kong                20.00
13                               Cliffs of Marineland                 6.81
14                                     Kirkeby Center                 6.81
15                              Gateway East Building                 6.81
16                                        Wayne Manor                 6.81
17                                      Bronson Caves                 6.81
18                    Royal Liver Building, Liverpool                 8.81
19  Ferruccio Lamborghini Museum, Funo di Argelato...                10.75
20  Lamborghini Museum 'MUDETEC', Sant'Agata Bolog...                10.77
21   Lamborghini Factory, Sant'Agata Bolognese, Italy                10.77
22                  Ferrari Gallery, Maranello, Italy                10.73
23   Pagani Automobili, San Cesario sul Panaro, Italy                10.72
24       SSC North America, Richland, Washington, USA                 6.36

Final answer:                                              Location  Travel Time (hours)
0          Necropolis Cemetery, Glasgow, Scotland, UK                 8.60
1           St. George's Hall, Liverpool, England, UK                 8.81
2   Millennium Mills, Royal Victoria Dock, East Lo...                 9.18
3   Knebworth House, Knebworth, Hertfordshire, Eng...                 9.14
4   Acton Lane Power Station, Acton Lane, Acton, L...                 9.16
5                                        Arctic World                 9.83
6                          Arkham Asylum (Nolanverse)                 1.45
7                               Arkham State Hospital                 1.45
8                       Warner Bros studios, New York                 1.01
9                     Warner Bros studios, Pittsburgh                 1.74
10                                            Chicago                 2.68
11                                   Bedfordshire, UK                 9.10
12                                          Hong Kong                20.00
13                               Cliffs of Marineland                 6.81
14                                     Kirkeby Center                 6.81
15                              Gateway East Building                 6.81
16                                        Wayne Manor                 6.81
17                                      Bronson Caves                 6.81
18                    Royal Liver Building, Liverpool                 8.81
19  Ferruccio Lamborghini Museum, Funo di Argelato...                10.75
20  Lamborghini Museum 'MUDETEC', Sant'Agata Bolog...                10.77
21   Lamborghini Factory, Sant'Agata Bolognese, Italy                10.77
22                  Ferrari Gallery, Maranello, Italy                10.73
23   Pagani Automobili, San Cesario sul Panaro, Italy                10.72
24       SSC North America, Richland, Washington, USA                 6.36

[Step 3: Duration 5.69 seconds| Input tokens: 11,391 | Output tokens: 1,352]

In [16]:
result

,Location,Travel Time (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Millennium Mills, Royal Victoria Dock, East Lo...",9.18
3,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.14
4,"Acton Lane Power Station, Acton Lane, Acton, L...",9.16
5,Arctic World,9.83
6,Arkham Asylum (Nolanverse),1.45
7,Arkham State Hospital,1.45
8,"Warner Bros studios, New York",1.01
9,"Warner Bros studios, Pittsburgh",1.74


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [19]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- We need to find Batman filming locations worldwide.
- We need to calculate the travel time for a cargo plane to Gotham for each filming location.
- We need to find supercar factories with the same cargo plane transfer time to Gotham.

### 1.2. Facts to look up
- List of Batman filming locations worldwide.
  - Source: Web search for "Batman filming locations" and verify from reliable sources like IMDb, official movie 
websites, or reputable news articles.
- Coordinates of each Batman filming location.
  - Source: Geocoding services or verified maps and location databases.
- List of supercar factories worldwide.
  - Source: Web search for "supercar factories" and verify from reliable sources like automotive industry websites 
or reputable news articles.
- Coordinates of each supercar factory.
  - Source: Geocoding services or verified maps and location databases.
- Cruising speed of a typical cargo plane.
  - Source: Aviation industry standards or reliable aviation websites.

### 1.3. Facts to derive
- Travel time for a cargo plane from each Batman filming location to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of each filming location and 
Gotham.
- Travel time for a cargo plane from each supercar factory to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of each supercar factory and 
Gotham.

## 2. Plan
1. Perform a web search for "Batman filming locations" and compile a list of locations.
2. For each Batman filming location, perform a web search to find its coordinates.
3. Perform a web search for "supercar factories" and compile a list of factories.
4. For each supercar factory, perform a web search to find its coordinates.
5. Perform a web search to find the cruising speed of a typical cargo plane.
6. Use the `calculate_cargo_travel_time` function to calculate the travel time from each Batman filming location to
Gotham.
7. Use the `calculate_cargo_travel_time` function to calculate the travel time from each supercar factory to 
Gotham.
8. Create a pandas dataframe with Batman filming locations and their travel times to Gotham.
9. Create a pandas dataframe with supercar factories and their travel times to Gotham.
10. Finalize the dataframes and prepare them for the final answer.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  supercar_factories = web_search(query="supercar factories")                                                      
                                                                                                                   
  print("Batman filming locations:", batman_locations)                                                             
  print("Supercar factories:", supercar_factories)                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman filming locations: ## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•87 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: Millennium Mills, Royal Victoria Dock, East London. The Batman location: Lt Gordon and
The Batman interrogate Penguin after the ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) · 21 ; Acton 
Lane Power Station, Acton Lane, Acton, London, ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [Category:Film Locations - Batman Wiki](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki | Fandom

A · Arctic World · Arkham Asylum (Nolanverse) · Arkham Asylum (Schumacherverse) · Arkham State Hospital ...

5. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

6. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

7. [The Dark Knight | 2008](https://movie-locations.com/movies/d/Dark-Knight.php)
Source: The Worldwide Guide To Movie Locations

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

8. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOopjfkJvYecQNFzxnPCBdnhxclZxuGeEmXi7ncu
u9CofWVDnJYeF)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

9. [The Filming Locations of The Batman : 
r/Tartaria](https://www.reddit.com/r/Tartaria/comments/tsd5zf/the_filming_locations_of_the_batman/)
Source: Reddit · r/Tartaria

The Filming Locations of The Batman · r/Tartaria - Royal Liver Building, Liverpool. Royal Liver Building, Liverpool
· r/Tartaria - Royal Liver ...
Supercar factories: ## Search Results
0. [Vroom! Touring Italy's Supercar 
Factories](https://www.nytimes.com/2025/03/14/travel/motor-valley-italy-supercars.html)
Date published: Mar 14, 2025
Source: The New York Times

In the country's “Motor Valley,” racecar enthusiasts can admire, and even drive, Maseratis, Lamborghinis, Ferraris 
and more.

1. [Car Factory Tours Italy](https://www.italianfactorymotortour.com/car-factory-tours-italy.html)
Source: Italian Factory Motor Tour

We offers Car Factory Tours Italy to the most famous car and bike production places and museums like Ferrari, 
Maserati, Lamborghini, Pagani and Ducati.

2. [GTM Supercar - Factory Five Racing](https://www.factoryfive.com/gtm-supercar/)
Source: Factory Five Racing

The GTM is our flagship supercar, and has come to symbolize our technological abilities. The car is drop-dead 
gorgeous and it runs with the very fastest ...

3. [Italian Supercars](htt

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(final_df)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer:                                                                       Location  \
Batman Filming Locations 0   Acton Lane Power Station, Acton Lane, Acton, L...   
                         1              Cliffs of Marineland, Los Angeles, USA   
                         2                     Bronson Caves, Los Angeles, USA   
                         3                                    Hong Kong, China   
                         4   Millennium Mills, Royal Victoria Dock, East Lo...   
                         5          Necropolis Cemetery, Glasgow, Scotland, UK   
                         6                                     Pittsburgh, USA   
                         7                 Royal Liver Building, Liverpool, UK   
                         8                            Batman, Los Angeles, USA   
                         9                                    Los Angeles, USA   
                         10                                   Bedfordshire, UK   
                         11  Knebworth House, Knebworth, Hertfordshire, Eng...   
                         12                                       Chicago, USA   
                         13                   Kirkeby Center, Los Angeles, USA   

                             Travel Time (hours)  
Batman Filming Locations 0                  9.16  
                         1                  6.81  
                         2                  6.78  
                         3                 19.99  
                         4                  9.19  
                         5                  8.60  
                         6                  1.74  
                         7                  8.81  
                         8                 14.33  
                         9                  6.77  
                         10                 9.11  
                         11                 9.15  
                         12                 2.68  
                         13                 6.77  

[Step 8: Duration 1.65 seconds| Input tokens: 78,608 | Output tokens: 5,864]

                                                                      Location  \
Batman Filming Locations 0   Acton Lane Power Station, Acton Lane, Acton, L...   
                         1              Cliffs of Marineland, Los Angeles, USA   
                         2                     Bronson Caves, Los Angeles, USA   
                         3                                    Hong Kong, China   
                         4   Millennium Mills, Royal Victoria Dock, East Lo...   
                         5          Necropolis Cemetery, Glasgow, Scotland, UK   
                         6                                     Pittsburgh, USA   
                         7                 Royal Liver Building, Liverpool, UK   
                         8                            Batman, Los Angeles, USA   
                         9                                    Los Angeles, USA   
                         10                                   Bedfordshire, UK   
                

In [20]:
detailed_report

Location  \
Batman Filming Locations 0   Acton Lane Power Station, Acton Lane, Acton, L...   
                         1              Cliffs of Marineland, Los Angeles, USA   
                         2                     Bronson Caves, Los Angeles, USA   
                         3                                    Hong Kong, China   
                         4   Millennium Mills, Royal Victoria Dock, East Lo...   
                         5          Necropolis Cemetery, Glasgow, Scotland, UK   
                         6                                     Pittsburgh, USA   
                         7                 Royal Liver Building, Liverpool, UK   
                         8                            Batman, Los Angeles, USA   
                         9                                    Los Angeles, USA   
                         10                                   Bedfordshire, UK   
                         11  Knebworth House, Knebworth, Hertfordshire, Eng...   
                         12                                       Chicago, USA   
                         13                   Kirkeby Center, Los Angeles, USA   

                             Travel Time (hours)  
Batman Filming Locations 0                  9.16  
                         1                  6.81  
                         2                  6.78  
                         3                 19.99  
                         4                  9.19  
                         5                  8.60  
                         6                  1.74  
                         7                  8.81  
                         8                 14.33  
                         9                  6.77  
                         10                 9.11  
                         11                 9.15  
                         12                 2.68  
                         13                 6.77

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [23]:
model = InferenceClientModel(
    model_id="Qwen/Qwen3-4B", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serpapi"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [24]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [54]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    # model=InferenceClientModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    # model=InferenceClientModel("deepseek-ai/DeepSeek-R1-Distill-Llama-70B", provider="together", max_tokens=8096),
    model = InferenceClientModel(
    model_id="deepseek-ai/DeepSeek-R1-0528",  # Stable + Together-supported
    provider="together",
    max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [55]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1-0528
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen3-4B
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                  

In [56]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ InferenceClientModel - deepseek-ai/DeepSeek-R1-0528 ───────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
<think>
We are given a task that involves several parts:
1. Find all Batman filming locations in the world.
2. Calculate the time to transfer via cargo plane from Gotham (40.7128° N, 74.0060° W) to each of these locations.
3. Also find some supercar factories that have the same cargo plane transfer time (i.e., the same travel time from 
Gotham) as one of the Batman filming locations. We need at least 6 points in total (so if we have fewer than 6 
Batman locations, we will need to find enough supercar factories to make up 6 or more).
4. Represent all these locations (Batman filming locations and supercar factories) on a spatial map as scatter 
points, with color depending on the travel time, and save the map to `saved_map.png`.

We have the following tools:
- `calculate_cargo_travel_time(origin_coords, destination_coords, cruising_speed_kmh)`: to compute travel time 
between two points (using great-circle distance). We can assume a cruising speed (default 750 km/h) if not 
specified.
- `web_agent(task, additional_args)`: to look up information on the web.
- `final_answer(fig)`: to return the final map.

We also have an example of how to plot the map using Plotly.

Let's break down the facts we have and need.

### 1.1. Facts given in the task
- We are in Gotham at coordinates (40.7128° N, 74.0060° W). Note: This is the same as New York City, but we'll use 
it as given.
- We need at least 6 points in total (Batman filming locations plus supercar factories).
- The example code for plotting uses `plotly.express` and saves the image.

### 1.2. Facts to look up
We need to gather two sets of locations:
1. Batman filming locations: We need a list of locations (with names and coordinates) where Batman movies (any in 
the franchise) have been filmed.
   - We can use the web_agent to search for "Batman filming locations around the world" or "list of Batman movie 
filming locations". We need to get the coordinates (latitude and longitude) for each location.

2. Supercar factories: We need a list of factories that manufacture supercars (e.g., Ferrari, Lamborghini, etc.) 
and their coordinates. However, note that we don't want all of them, but only those that have the same travel time 
from Gotham as one of the Batman filming locations. But we don't know the travel times until we compute them for 
the Batman locations. So we might first get a list of supercar factories and then compute their travel times and 
then match.

   However, the task says: "Also give me some supercar factories with the same cargo plane transfer time." This 
implies that we need to find supercar factories that have a travel time that matches one of the travel times of the
Batman locations. But note: we need at least 6 points in total. So if we have 3 Batman locations, we need at least 
3 supercar factories that have travel times matching one of the three.

   But how to look up supercar factories? We can use the web_agent to search for "list of supercar manufacturers" 
or "supercar factories around the world" and then get their headquarters or factory locations and coordinates.

   However, note: we are to find factories that have the same travel time. So we might have to compute the travel 
time for each Batman location and then for each supercar factory and then select those that match. But note: travel
time is a continuous value, so we might have to allow a small tolerance.

   Alternatively, we can note that the task says "with the same cargo plane transfer time" meaning we are to find 
supercar factories that have a travel time equal to one of the Batman locations. But if we have multiple Batman 
locations, we might have multiple travel times. We are to find supercar factories that have a travel time that 
matches any of the Batman locations.

   However, the

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
We are going to break down the task into steps and use the web_agent for the two lookups (Batman locations and     
supercar factories). We assume that the web_agent will return a list of dictionaries with keys: 'name', 'lat',     
'lon'. If the web_agent returns addresses instead of coordinates, we would need to geocode, but we'll assume it    
returns coordinates.                                                                                               
                                                                                                                   
However, note: the web_agent might return a string or a structured data? The tool says: "Browses the web to find   
information" and returns a string. So we have to parse the string? But the problem says: "Never try to process     
strings using code: when you have a string to read, just print it and you'll see it." So we are to print the result
and then use it? But we are in code. We cannot parse the string arbitrarily? We have to rely on the web_agent      
returning a structured format? Or we can ask the web_agent to return in a specific format.                         
                                                                                                                   
We can specify in the task to the web_agent to return a JSON string? But the tool returns a string. We can ask the 
web_agent to return a list in JSON [38;2;230;237;243;48;2;13;17;

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
                Here is your code snippet:
                <think>
We are going to break down the task into steps and use the web_agent for the two lookups (Batman locations and 
supercar factories). We assume that the web_agent will return a list of dictionaries with keys: 'name', 'lat', 
'lon'. If the web_agent returns addresses instead of coordinates, we would need to geocode, but we'll assume it 
returns coordinates.

However, note: the web_agent might return a string or a structured data? The tool says: "Browses the web to find 
information" and returns a string. So we have to parse the string? But the problem says: "Never try to process 
strings using code: when you have a string to read, just print it and you'll see it." So we are to print the result
and then use it? But we are in code. We cannot parse the string arbitrarily? We have to rely on the web_agent 
returning a structured format? Or we can ask the web_agent to return in a specific format.

We can specify in the task to the web_agent to return a JSON string? But the tool returns a string. We can ask the 
web_agent to return a list in JSON format? Then we can parse it.

Alternatively, we can assume that the web_agent returns a string that we can parse. But the problem says not to 
process strings? So we have to rely on the web_agent returning a string that we can print and then use? But we are 
in a multi-step process. We can print the string and then in the next step, we can manually extract the data? But 
that would require human intervention? We are an automated agent.

We must design the task for the web_agent to return the data in a structured way? We can try:

In the task description for the web_agent, we can say: "Return the list as a JSON string that can be parsed in 
Python, with the following structure: [{'name': 'Location Name', 'lat': 12.34, 'lon': 56.78}, ...\]"

But note: the web_agent is a human? The tool says: "Since this team member is a real human, be as detailed and 
verbose as necessary in your task description." So we can ask for a specific format.

We'll do that.

So for step 1:

Task for Batman locations:
"Search for a comprehensive list of filming locations for Batman movies (all movies in the franchise). For each 
location, provide the name of the place, the city or country if necessary, and the latitude and longitude 
coordinates. If the coordinates are not available, provide the address so that we can geocode, but prefer 
coordinates. Format the result as a JSON string that can be parsed in Python, with the following structure: a list 
of dictionaries, each dictionary has keys: 'name' (string), 'lat' (float, if available), 'lon' (float, if 
available). If only an address is available, then use a key 'address' (string) and leave out 'lat' and 'lon'. We 
need at least the most famous ones."

But note: if we get an address, we would have to geocode it? We don't have a geocoding tool. So we must avoid that.
We'll ask for coordinates.

Alternatively, we can ask the web_agent to geocode? But the web_agent is a human, so they can do that? We can say: 
"If coordinates are not available, please geocode the address to get the latitude and longitude and then include 
only the coordinates."

So we'll adjust the task.

Similarly for supercar factories.

But note: the web_agent might not be able to return a JSON string? It might return a string that is a JSON 
representation? Then we can parse it with `json.loads`.

We'll do:

Step 1: Call web_agent for Batman locations and then parse the result as JSON.

But what if the web_agent returns a non-JSON string? Then we have to handle it. We'll assume the web_agent follows 
the instruction.

Similarly for step 3.

Now, let's write the code step by step.

We'll do:

Step 1: Get Batman locations via web_agent and parse the result.

Step 2: For each location, if it has an address (but we as

...

Code execution failed at line 'import plotly.express as px' due to: InterpreterError: Import of plotly.express is 
not allowed. Authorized imports are: ['statistics', 'json', 'unicodedata', 're', 'itertools', 'collections', 
'time', 'shapely', 'datetime', 'pandas', 'geopandas', 'math', 'plotly', 'random', 'stat', 'numpy', 'queue'\]

[Step 13: Duration 6.26 seconds| Input tokens: 184,986 | Output tokens: 10,108]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 14 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 14: Duration 5.39 seconds]

KeyboardInterrupt: 

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)